# NLP

Natural Language Processing, 자연어처리

‘자연어처리의 상업용 응용(Commercial Application of Natural Language Processing)’
1. 자연어 이해
2. 자연어 생성
3. 음성인식
4. 기계번역
5. 오타 검열

## 언어모델의 발전
1. 합성곱신경망 (CNN, Convolutional Neural Network)
2. 순환신경망 (RNN, Recurrent Neural Network): 결과값을 다음 단어의 입력층에도 전달
3. seq2seq 언어모델
    - 문장을 차례로 생성해내는 모델
    - RNN을 이용한 인코더-디코더의 조합
    - 번역될 문장과 번역된 문장을 조건부 확률의 문제로 해석 
    - 이전 입력을 기억하는 RNN이지만 중심 단어와 거리가 멀어지면 성능이 저하되어 Attention 매커니즘이 탄생
4. Attention: 이전의 각 단어의 결과값(hidden_state)를 한꺼번에 참고하여 디코더에 단어가 생성되는 순간에 중요도가 높은 인코더의 hidden_state를 중요하게 참조하는 구조, 중요한 단어는 모델이 스스로 학습하며 알아냄
    - RNN으로 이전 단어의 정보를 참고하지만 Attention으로 문장 전체 특징을 볼 수 있게 됨.
    - 그럼 RNN 필요없는거 아니냐? -> Transformer 탄생
5. Transformer: RNN을 제거하고 self-attention만을 강조한 구조
    - Multi-headed Attention: 여러개의 인코더를 쌓아올린 구조가 적용되어 큰 성능 향상으로 이어짐
    - 각 단어 임베딩과 각 단어의 순서를 알기 위해 positional embedding을 추가하여 문장을 한 번에 연산 => 학습속도 
    - "Attention Is All You Need"


In [ ]:
import torch
import torch.nn as nn

transformer_model = nn.Transformer(nhead = 16, num_encoder_layers = 12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

print(f"src: {src}")
print(f"tgt: {tgt}")
print(f"> Transformer.\n{transformer_model}")
print(f"out: {out}")

![](transformer_architecture.jpg)
 PyTorch의 nn.Transformer는 가운데 구조만 정의함.Linear, Positional Encoding, Embedding 등은 직접 따로 구현해야 한다. [참고](https://wikidocs.net/156986)
 
[HuggingFace의 Transformers 강좌](https://wikidocs.net/book/8056)

# BERT, Bidirectional Encoder Representations from Transformers

- 기존 언어모델은 앞의 단어들을 참조하여 다음에 나올 단어를 예측하는 방식
- BERT는 문장의 중간 단어를 마스킹한 후 전체 문장에서 해당 단어를 예측하는 방식으로 학습되는 MLM(Masked Language Model)
- 두 문장이 이어지는 관계인지 아닌지를 학습하는 NSP(Next Sentence Prediction) 기능을 추가
- 문장을 생성하지 않고 **문장을 분석하고 이해하는데만 집중하는 모델**로 Transformer 구조에서 디코더를 생략하고 인코더만 이용했다.
- 이런 방식으로 많은 데이터를 학습하여 하이퍼 파라미터 값을 생성했더니, 각각의 독립적인 분류, 추론, 문장비교, 질문대답 등의 task에서 간단한 레이어를 추가하고, 적은 데이터와 학습시간으로 fine-tuning만 거쳐도 기존의 각 task별 SOTA(현 최고성적) 모델들을 압도하는 성능을 보여주었으며 이를 전이학습(Transfer Learning)이라 부른다.
- 굉장히 거대한 모델이라 구글의 TPU를 사용해도 수일에서 수주가 걸리는 학습시간이 필요하나 미리 학습된 모델과 소스코드까지 오픈소스로 공개되어있다.

## BERT의 구조

1. 입력 Sentence
2. Tokenizer
    - Token: 입력 Sentence를 토큰화, 문장 시작은 CLS, 문장 끝은 SEP, 그 외 특수토큰을 포함할 수 있다.
    - Segment: 첫 번째 문장은 0, 두 번째 문장은 1
    - Position: 각 토큰의 위치를 표시, Transformer의 Positional Embedding과는 다름.
3. BERT: 위 3가지를 합산하여 입력으로 전달됨

    | model | hidden layer | hidden size | multi-head |  parameter   | device | training days |
    |-------|-------------:|------------:|-----------:|:------------:|-------:|:-------------:|
    | base  |           12 |         768 |         12 | 110M(1억 1천만) |  4 TPU |    4 days     |
    | large |           24 |        1024 |         16 | 340M(3억 4천만) | 16 TPU |    4 days     |

4. Output: BERT의 출력벡터, 적당한 레이어를 추가하여 fine-tuning으로 원하는 task에 적용하게 됨.

## BERT의 학습

- 입력에 대하여 15%내의 토큰을 masking하고 모델이 내부 양방향 연산을 통해 적절한 토큰을 예측하여 문장을 완성 후 실제 단어와 비교하며 학습
- 두 문장일 경우 뒤 문장이 앞 문장의 연결문장인지 여부를 판단하여 학습
- 입력 시퀀스의 토큰은 최애 512개
- batch-size: 256 시퀀스 * 512 토큰 = 131,072 토큰/batch 1M STEP => 40 epochs
- learning rate: 1e-4, β1 = 0.9, β2= 0.999 

## BERT의 성능을 높인 기술들

1. RoBERTa(A Robustly Optimized BERT Pretraining Approach)
    - 더 많은 데이터를 사용하여 더 오래, 더 큰 batch로 학습
    - NSP 제거
    - 짧은 sequence는 배제하고 더 긴 sequence로 학습
    - static masking을 dynamic masking으로 바꿈

2. ALBERT(A Lite BERT)
    - input token embedding 사이즈를 줄여 전체 파라미터를 줄임
    - Transformer Layer 간 같은 Parameter를 공유
    - NSP 대신 SOP(Sentence Order Prediction) 사용

3. DistilBERT(a distilled version of BERT)
    - KD(Knowledge Distillation)라는 압축기술 사용, 큰 모델(선생)과 작은 모델(학생)을 학습시킴
    - 크기는 40% 감소, 속도는 60% 증가, 결과치는 97% 유지
    - 학생은 선생의 출력결과의 확률 분포를 배움으로써 복잡한 특징 신호를 단순한 구조로도 배울 수 있게 되어 성능 하락이 거의 없이 빠르다.
    - ALBERT는 구조를 줄인 형태이고 DistilBERT는 이미 사전학습된 BERT를 압축하는 구조

4. BART(Bidirectional Auto-Regressive Transformer)
    - BERT 방식에 GPT의 디코더 구조를 사용해 나머지 성능은 유지하고, 문장 생성, 지문 해석 등에서 큰 성능향상을 보여준다.
    - Denoising Autoencoder 방식으로 사전학습되는데 BERT처럼 noise를 복구하는 방식으로 학습한다.
    - 어떤 형태의 변형(noise)이던 적용 가능하다.
        - Token Masking
        - Sentence Permutation
        - Document Rotation
        - Token Deletion
        - Test Infilling
    - 6개의 인코더와 6개의 디코더 레이어를 가지며 인코더에서 디코더로 넘어갈 때 cross-attention을 수행
    - BERT보다 약 10% 많은 파라미터를 갖는다.

5. ELECTRA(Efficiently Learning an Encoder that Classifies Token Replacements Accurately)
    - BERT 학습의 효율성에 기반하여 새로운 pre-training 방식을 제시한 버전으로, MLM 대신 RTD(Replaced Token Detection) 사용
    - 샘플 한 문장을 부분적으로 단어가 바뀐 여러 문장으로 생성이 가능하므로 원본 문장의 전체 단어 모두를 사용 가능해 데이터셋의 크기 대비 훈련 효율을 향상시킨다.

# GPT
Generative Pre-trained Transformer

- Transformer 구조에서 인코더는 무시하고 디코더 부분만 집중하여 만든 문장생성모델
- 대량의 문서를 학습하여 어떤 단어가 주어졌을 때 다음에 올 확률이 가장 높은 단어를 제시하여 순차적으로 문장을 만듦

# XLNet

- BERT가 각 masking 단어별로 독립적이라고 가정하여 생기는 단어간의 위치에 대한 연관성이 무시되는 문제와 fine-tuning 시의 masking 기법이 쓰이지 않음으로 부조화에 의한 성능저하를 극복하고자 나온 방식
- masking을 쓰지 않고 단어를 뒤섞어서 순차적으로 예측하는 방식을 여러번 반복하여 학습하는 Permutation Language Model
- 이후 fine-tuning 시에도 기법의 부조화가 없어서 성능향상이 된다.
- 실제 작동은 단어를 뒤섞지 않고 Transformer의 self-attention을 활용하여 attention-mask로 구현된다.

# T5
Text-to-Text Transfer Transformer

- BERT가 분류나 입력 범위만 출력할 수 있는 것에 반해 모든 입력과 출력을 문자열로 하고, 각각의 task 역시 입력 문자열에 포함하여 재구성한 모델
- 훈련을 위해 wikipedia보다 2배 큰 C4(Colossal Clean Crawled Corpus)라는 잘 정제되고 다양한 새로운 훈련 데이터셋을 만들어 훈련하고 공개
- 알려진 NLP의 다양한 task 뿐만 아니라 새로운 유형의 task에도 쉽게 적응하며 뛰어난 성능을 보여준다.

# kpfBERT (언론진흥재단 BERT)

## 신문기사에 특화된 kpfBERT)
언론에 특화된 사전학습된 BERT 모델을 생성함으로써 여러 언론 관련 사업에서 자연어처리 작업에 활용할 수 있는 기반을 만들었다.

## kpfBERT의 활용
- [뉴스 본문 요약](https://github.com/KPFBERT/kpfbertsum)
- [kpf-SBERT](https://github.com/KPFBERT/kpfSBERT)
    - Sentence BERT를 제작하는 방법에 대한 소스코드
    - 문장 전체를 동일한 크기의 임베딩 벡터로 변환해 문장간의 비교를 BERT 대비 빠르고 효율적으로 연산할 수 있다.
- [kpf-SBERT를 이용한 뉴스 클러스터링](https://github.com/KPFBERT/kpfSBERT_Clustering): HDBSCAN으로 뉴스 자동 클러스터링하는 모델의 예제
- 그 외
    - 맞춤법 검사기
    - 단어 자동완성
    - 문장의 어색한 표현이나 어휘 체크
    - 혐오표현 순화
    - 기사의 논조 분석
    - 광고성 기사 검출
    - 그 외